In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import os
import re
import numpy as np
import pandas as pd
import collections

In [2]:
x_name = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터3] samp_cst_feat'}.csv")
df_x = pd.read_csv(x_name, index_col=0)

y_name = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터2] samp_train'}.csv")
df_y = pd.read_csv(y_name, index_col=0)

df = pd.merge(df_x, df_y, on='cst_id_di')

In [3]:
df.MRC_ID_DI[df.MRC_ID_DI > 0 ] = 1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [4]:
nc_name = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터4] variable_dtype'}.xlsx")
nc = pd.read_excel(nc_name, index_col=0)

In [ ]:
df

In [ ]:
%matplotlib inline
plt.figure(figsize = (227, 227))
sns.heatmap(df.corr(), annot = True, fmt = '.2f', square = True)
plt.savefig('corr13.png')
plt.show()

In [ ]:
pd.set_option('display.max_columns', None)
df.describe([.01, .10, .20, .30, .40, .50, .60, .70, .80, .90, .95, .99])

In [ ]:
for i in df.columns[0:]:
    if i == 'MRC_ID_DI' or nc.loc[i, 'dType'] == 'categorical':
        ax = sns.countplot(df[i], palette=['#432371',"#FAAE7B"])
        plt.show()
    else:
        ax = sns.boxplot(x=i, data=df, color = 'yellow')
        ax.set(xlim=(-1, 1))
        plt.show()

In [ ]:
#https://www.pluralsight.com/guides/cleaning-up-data-from-outliers
#https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba
Q1 = df.quantile(0.05)
Q3 = df.quantile(0.95)
IQR = Q3 - Q1
df_out = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]
print(df_out.shape)

In [ ]:
#https://www.kite.com/python/answers/how-to-remove-outliers-from-a-pandas-dataframe-in-python
from scipy import stats
import numpy as np

z_scores = stats.zscore(df)

abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 5).all(axis=1)
df = df[filtered_entries]
print(df.shape)

In [5]:
"""
drop_columns = list()
for i in df.columns[0:-1]:
    if nc.loc[i, 'dType'] == 'numerical':
        if df[i].quantile(0.75) - df[i].quantile(0.25) > 0.5:
            print(i)
            drop_columns.append(i)
    else:
        if len(df[df[i] == 1]) > len(df)*0.8 or len(df[df[i] == 0]) > len(df)*0.8:
            drop_columns.append(i)

df = df.drop(columns = drop_columns, axis=1)
"""
#https://www.pluralsight.com/guides/cleaning-up-data-from-outliers
for i in df.columns[0:-1]:
    if nc.loc[i, 'dType'] == 'numerical':
        d_90 = df[i].quantile(0.99)
        d_10 = df[i].quantile(0.01)
        d_50 = df[i].quantile(0.50)
        df[i] = np.where(df[i] > d_90, d_90, df[i])
        df[i] = np.where(df[i] < d_10, d_10, df[i])
print(df.shape)

(10124, 227)


In [6]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import random

physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

seed_value= 0
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

In [7]:
import sys
ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                      exception_only=False, running_compiled_code=False):
       etype, value, tb = sys.exc_info()
       return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback

In [8]:
df_0 = df[df['MRC_ID_DI'] == 0].sample(frac=1)
df_1 = df[df['MRC_ID_DI'] == 1].sample(frac=1)

sample_size = len(df_0) if len(df_0) < len(df_1) else len(df_1)

df_h = pd.concat([df_0.head(sample_size), df_1.head(sample_size)]).sample(frac=1)
df_t = pd.concat([df_0.tail(sample_size), df_1.head(sample_size)]).sample(frac=1)
df_f = pd.concat([df_h, df_t]).sample(frac=1)

In [9]:
X = df_f.drop(columns = ['MRC_ID_DI', 'VAR021', 'VAR046'], axis=1)
y = tf.keras.utils.to_categorical(df_f['MRC_ID_DI'])

In [10]:
num, cat = list(), list()
for i in X.columns:
    if nc.loc[i, 'dType'] == 'numerical':
        num.append(i)
    else:
        if len(df[df[i] == 1]) > len(df)*0.8 or len(df[df[i] == 0]) > len(df)*0.8:
            continue
        else:
            #cat.append(i)
            num.append(i)
X = X[num]

for i in X.columns:
    if nc.loc[i, 'dType'] == 'numerical':
        if 2*X[i].quantile(1) < 1 and  2*X[i].quantile(0) > -1:
            X[i] *= 2
        
#X[X.select_dtypes(include=['number']).columns] *= 10

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
"""
X1_train = X_train[num]
X1_test = X_test[num]

X2_train = X_train[cat]
X2_test = X_test[cat]


from sklearn import preprocessing

x = X1_train.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
X1_train = pd.DataFrame(x_scaled)

x = X1_test.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
X1_test = pd.DataFrame(x_scaled)
"""

In [ ]:
"""
input_1 = tf.keras.Input(dtype = tf.float32, shape = (len(num),))
input_2 = tf.keras.Input(dtype = tf.float32, shape = (len(cat),))

dense_layer_1_1 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(input_1)
dense_layer_1_2 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_1_1)
dense_layer_1_3 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_1_2)
dense_layer_1_4 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_1_3)
dropout_1_5 = tf.keras.layers.Dropout(rate = 0.2)(dense_layer_1_1)


dense_layer_2_1 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(input_2)
dense_layer_2_2 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_2_1)
dense_layer_2_3 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_2_2)
dense_layer_2_4 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_2_3)
dropout_2_5 = tf.keras.layers.Dropout(rate = 0.2)(dense_layer_2_1)

concat_layer = tf.keras.layers.Concatenate()([dropout_1_5, dropout_2_5])


dense_layer_3 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(concat_layer)
dense_layer_4 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_3)
dense_layer_5 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_4)
dense_layer_6 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_5)

output = tf.keras.layers.Dense(units = 2, activation = tf.nn.softmax)(dense_layer_3)
"""

In [ ]:
input_ = tf.keras.Input(dtype = tf.float32, shape = (len(X.columns),))

dense_layer_1_1 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(input_)
dense_layer_1_2 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_1_1)
dense_layer_1_3 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_1_2)
dense_layer_1_4 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_1_3)

dropout_1_5 = tf.keras.layers.Dropout(rate = 0.2)(dense_layer_1_1)

output = tf.keras.layers.Dense(units = 2, activation = tf.nn.softmax)(dropout_1_5)

In [ ]:
model = tf.keras.Model(inputs=input_, outputs=output)
#model = tf.keras.Model(inputs=[input_1, input_2], outputs=output)

In [ ]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=0.00005), metrics=['acc'])
print(model.summary())

In [ ]:
#history = model.fit(x=X_train, y=y_train, batch_size=32, epochs=400, verbose=1, validation_split=0.1)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100)
history = model.fit(x=X_train, y=y_train, batch_size=32, epochs=10000, verbose=1, validation_split=0.1, callbacks=[callback])
#history = model.fit(x=[X1_train, X2_train], y=y_train, batch_size=32, epochs=2000, verbose=1, validation_split=0.1, callbacks=[callback])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])

plt.title('model accuracy')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

In [ ]:
score = model.evaluate(x=X_test, y=y_test, verbose=1)
#score = model.evaluate(x=[X1_test, X2_test], y=y_test, verbose=1)

print("Test Score:", score[0])
print("Test ACC:", score[1])

In [12]:
learning_rate = 0.00005
training_epochs = 700
batch_size = 32
num_models = 10

tf.compat.v1.disable_eager_execution()

class Model:

    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()

    def _build_net(self):
        with tf.compat.v1.variable_scope(self.name):
            
            self.training = tf.keras.backend.placeholder(dtype = tf.bool)

            self.X = tf.keras.backend.placeholder(dtype = tf.float32, shape=(None,len(X_train.columns)))            
            self.Y = tf.keras.backend.placeholder(dtype = tf.float32, shape=(None,2))

            dense1 = tf.compat.v1.layers.dense(inputs=self.X, units = 10, activation=tf.nn.relu)
            dropout1 = tf.compat.v1.layers.dropout(inputs=dense1, rate=0.2, training=self.training)
            
            self.logits = tf.keras.layers.Dense(units = 2, activation = tf.nn.softmax)(dropout1)

        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.Y))
        self.optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)

        correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def predict(self, x_test, training=False):
        return self.sess.run(self.logits,
                             feed_dict={self.X: x_test, self.training: training})

    def get_accuracy(self, x_test, y_test, training=False):
        return self.sess.run(self.accuracy,
                             feed_dict={self.X: x_test,
                                        self.Y: y_test, self.training: training})

    def train(self, x_data, y_data, training=True):
        return self.sess.run([self.cost, self.optimizer], feed_dict={
            self.X: x_data, self.Y: y_data, self.training: training})

    
sess = tf.compat.v1.Session()


models = []
for m in range(num_models):
    models.append(Model(sess, "model" + str(m)))

sess.run(tf.compat.v1.global_variables_initializer())

print('Learning Started!')


for epoch in range(training_epochs):
    avg_cost_list = np.zeros(len(models))
    total_batch = int(len(X_train)/ batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = X_train[i * batch_size:(i+1) * batch_size], y_train[i * batch_size:(i+1) * batch_size]
        for m_idx, m in enumerate(models):
            c, _ = m.train(batch_xs, batch_ys)
            avg_cost_list[m_idx] += c / total_batch
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', avg_cost_list)

print('Learning Finished!')


test_size = len(y_test)
predictions = np.zeros([test_size, 2])
for m_idx, m in enumerate(models):
    print(m_idx, 'Accuracy:', m.get_accuracy(
        X_test, y_test))
    p = m.predict(X_test) 
    predictions += p

ensemble_correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y_test, 1))
ensemble_accuracy = tf.reduce_mean(tf.cast(ensemble_correct_prediction,  tf.float32))
print('Ensemble accuracy:', sess.run(ensemble_accuracy))

dir = os.path.dirname(os.path.realpath(__file__))
all_saver = tf.train.Saver() 
all_saver.save(sess, dir + '/0909.h5')

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use keras.layers.dropout instead.
Learning Started!
Epoch: 0001 cost = [0.68504882 0.66365999 0.67844496 0.69870186 0.66266388 0.653229
 0.68538455 0.62645688 0.64887374 0.67107238]
Epoch: 0002 cost = [0.63721739 0.62378022 0.63103193 0.63210073 0.62764854 0.62973201
 0.63316552 0.59942172 0.61812893 0.6243505 ]
Epoch: 0003 cost = [0.61009197 0.60125751 0.60814976 0.60361609 0.61125207 0.6125706
 0.61194628 0.58710713 0.59900578 0.60374526]
Epoch: 0004 cost = [0.59622883 0.59075533 0.59550619 0.59012423 0.60029424 0.60163527
 0.59998824 0.58036843 0.58773755 0.59317397]
Epoch: 0005 cost = [0.58450569 0.58633616 0.58684664 0.58255499 0.59531001 0.59280514
 0.59222559 0.57391295 0.57969237 0.5848519 ]
Epoch: 0006 cost = [0.57981377 0.58048778 0.58123317 

Epoch: 0055 cost = [0.52020193 0.52119973 0.52161358 0.52378415 0.52882138 0.52450485
 0.53074886 0.52153293 0.524361   0.52330386]
Epoch: 0056 cost = [0.51954402 0.52219434 0.52215971 0.5250663  0.52876399 0.52443847
 0.529317   0.52004341 0.52420842 0.52282402]
Epoch: 0057 cost = [0.51956207 0.51879341 0.52042679 0.52506172 0.52748803 0.52386685
 0.52896987 0.52135502 0.52340838 0.52274295]
Epoch: 0058 cost = [0.52036412 0.52208503 0.52114453 0.52420227 0.52675897 0.52222951
 0.5299973  0.52134443 0.52163537 0.52025327]
Epoch: 0059 cost = [0.52054797 0.52029869 0.52035276 0.52593562 0.52635718 0.52260763
 0.52948988 0.51809313 0.52345338 0.52142773]
Epoch: 0060 cost = [0.51934397 0.519305   0.51930195 0.52473901 0.52842047 0.5201227
 0.52747302 0.51982122 0.52193557 0.52050338]
Epoch: 0061 cost = [0.51894759 0.51921607 0.52062376 0.52521067 0.52587768 0.52209492
 0.5295428  0.51823966 0.52125845 0.52033435]
Epoch: 0062 cost = [0.5196662  0.51877679 0.51976179 0.52353757 0.5262972  0.

Epoch: 0118 cost = [0.50860792 0.50544095 0.50970302 0.51318632 0.51203379 0.51037592
 0.51594725 0.50956031 0.51054312 0.50852529]
Epoch: 0119 cost = [0.50926015 0.5080222  0.50853328 0.51320573 0.51288609 0.50999121
 0.51415104 0.50874027 0.51162826 0.50829307]
Epoch: 0120 cost = [0.51098895 0.50674048 0.50785793 0.51105343 0.51171355 0.50961187
 0.51429208 0.50871847 0.51154814 0.50845255]
Epoch: 0121 cost = [0.50835377 0.50832052 0.50809254 0.51230825 0.51280502 0.50965258
 0.5124712  0.50846459 0.51306437 0.50799574]
Epoch: 0122 cost = [0.50908599 0.50630731 0.50781298 0.51255448 0.51217649 0.51013133
 0.51286119 0.50962424 0.51162842 0.50723666]
Epoch: 0123 cost = [0.50891724 0.50706566 0.50877221 0.51043404 0.50979231 0.5100631
 0.51256604 0.50943536 0.51097043 0.50769353]
Epoch: 0124 cost = [0.50673785 0.5073861  0.50763807 0.51087611 0.50928412 0.50897002
 0.51192417 0.50860425 0.511001   0.50782691]
Epoch: 0125 cost = [0.50653568 0.50599252 0.50891873 0.51008263 0.51153382 0.

Epoch: 0181 cost = [0.50271221 0.50085505 0.50238097 0.5040902  0.50360025 0.50252787
 0.50729267 0.50316148 0.5069353  0.50286893]
Epoch: 0182 cost = [0.50257218 0.50222281 0.50118206 0.50361597 0.50516985 0.50321818
 0.50360828 0.50332326 0.50478229 0.50093378]
Epoch: 0183 cost = [0.5019521  0.50061148 0.50230131 0.50267757 0.50339831 0.50310512
 0.50578173 0.50394018 0.5033496  0.50204852]
Epoch: 0184 cost = [0.50066317 0.50147409 0.50226199 0.50440866 0.50506973 0.50325623
 0.50494024 0.50295392 0.50417856 0.50273898]
Epoch: 0185 cost = [0.50154237 0.50077362 0.50025212 0.50321354 0.50312618 0.50285728
 0.50413285 0.50224389 0.5062867  0.5020711 ]
Epoch: 0186 cost = [0.50244427 0.50095917 0.50289655 0.50421024 0.50240524 0.50360665
 0.50467955 0.5027489  0.50386338 0.50267671]
Epoch: 0187 cost = [0.50249979 0.49960407 0.50020809 0.50315002 0.50163553 0.5028548
 0.50342361 0.5022638  0.50316173 0.49850695]
Epoch: 0188 cost = [0.50119604 0.50024849 0.50034941 0.50361484 0.50183933 0.

Epoch: 0244 cost = [0.4974771  0.49552555 0.49677058 0.49633146 0.49834475 0.49837074
 0.49857588 0.49876352 0.50030638 0.49704825]
Epoch: 0245 cost = [0.49600974 0.49678347 0.49598273 0.49729644 0.49684674 0.49812556
 0.49685788 0.49807085 0.50156261 0.4959493 ]
Epoch: 0246 cost = [0.49687121 0.49629803 0.49598684 0.49658018 0.49776701 0.49927681
 0.5006113  0.49900943 0.50013213 0.49735138]
Epoch: 0247 cost = [0.49666611 0.49595322 0.49655487 0.49729933 0.49850215 0.49738849
 0.49805422 0.4972824  0.49794726 0.4962902 ]
Epoch: 0248 cost = [0.49773541 0.49541169 0.49623997 0.49697068 0.49785982 0.49796546
 0.4981348  0.49727662 0.50024689 0.49696217]
Epoch: 0249 cost = [0.49729841 0.49522166 0.4959454  0.49633781 0.49632798 0.49730205
 0.49764245 0.49802442 0.49903089 0.49619091]
Epoch: 0250 cost = [0.49673032 0.49415408 0.49417201 0.4970295  0.4980334  0.49667868
 0.49875412 0.49921117 0.49811684 0.49722312]
Epoch: 0251 cost = [0.49805253 0.49700693 0.49686544 0.49404117 0.4981247  0

...

Epoch: 0496 cost = [0.48144776 0.4815831  0.48223143 0.47914742 0.48202727 0.48403995
 0.48073305 0.48186596 0.48496562 0.48277805]
Epoch: 0497 cost = [0.48327382 0.48298938 0.48357198 0.47921189 0.48312886 0.48246045
 0.47963411 0.48346041 0.48444569 0.48231947]
Epoch: 0498 cost = [0.48121231 0.4832018  0.48168235 0.47845582 0.48303943 0.4819644
 0.48189699 0.4827973  0.48366868 0.48384052]
Epoch: 0499 cost = [0.48231175 0.48324282 0.48149751 0.48073613 0.48426716 0.48431243
 0.48106664 0.48346615 0.48472971 0.4835429 ]
Epoch: 0500 cost = [0.48179766 0.47977651 0.48148484 0.47748084 0.48195592 0.48221443
 0.48084973 0.4808979  0.4846625  0.48326217]
Epoch: 0501 cost = [0.48227869 0.47991121 0.48190841 0.47838232 0.48182347 0.48252039
 0.4815974  0.48091151 0.48404644 0.48276699]
Epoch: 0502 cost = [0.48194085 0.48080905 0.48173223 0.4787561  0.48313365 0.48316776
 0.48062678 0.48141316 0.48385132 0.48353809]
Epoch: 0503 cost = [0.48186161 0.48163365 0.48205688 0.47939316 0.48271343 0.

Epoch: 0559 cost = [0.47977877 0.47939427 0.47944024 0.47506348 0.48056192 0.4810857
 0.47715297 0.4805564  0.48330877 0.4805147 ]
Epoch: 0560 cost = [0.47884096 0.4794388  0.47998365 0.47424954 0.47913913 0.48205217
 0.47673116 0.47900431 0.48110278 0.47974241]
Epoch: 0561 cost = [0.47958222 0.48081824 0.48128237 0.47508201 0.47875194 0.4809022
 0.47840011 0.48005284 0.48080708 0.47860031]
Epoch: 0562 cost = [0.47748424 0.47846742 0.47920325 0.47536419 0.47987018 0.48135763
 0.47870358 0.47781962 0.48168784 0.47987598]
Epoch: 0563 cost = [0.47842474 0.48015065 0.4793598  0.47516535 0.48042644 0.47992939
 0.47722124 0.479381   0.4823941  0.48033015]
Epoch: 0564 cost = [0.47912072 0.48092359 0.48189444 0.47685982 0.47828016 0.4797564
 0.47797245 0.47786864 0.48135119 0.48113411]
Epoch: 0565 cost = [0.47935654 0.4798075  0.47951452 0.47571421 0.47982685 0.48001201
 0.47711283 0.47828689 0.48138146 0.48127108]
Epoch: 0566 cost = [0.48105278 0.48054538 0.47912982 0.47567986 0.47885458 0.47

Epoch: 0622 cost = [0.47819567 0.47635973 0.47724858 0.47358998 0.47471958 0.4781308
 0.47380287 0.4772827  0.47810904 0.47804522]
Epoch: 0623 cost = [0.47747161 0.47470192 0.47620686 0.472908   0.47743584 0.47720955
 0.4755078  0.47559795 0.47691002 0.4786093 ]
Epoch: 0624 cost = [0.47766421 0.47708876 0.47762858 0.47164223 0.47689429 0.47832909
 0.47603078 0.4756091  0.48112392 0.47747842]
Epoch: 0625 cost = [0.4754317  0.47610152 0.4776954  0.47250324 0.47682457 0.47991795
 0.47451279 0.47804265 0.47692713 0.47761591]
Epoch: 0626 cost = [0.47727713 0.4769817  0.47693687 0.47245529 0.47875135 0.47710254
 0.47505921 0.47596574 0.47859801 0.47794349]
Epoch: 0627 cost = [0.47548535 0.47692094 0.47797871 0.47176284 0.47603171 0.4795182
 0.47552884 0.47664211 0.47727299 0.4765908 ]
Epoch: 0628 cost = [0.47603141 0.47577063 0.47648582 0.47098111 0.47673961 0.47851907
 0.47678209 0.47644062 0.47884293 0.4761839 ]
Epoch: 0629 cost = [0.47598454 0.47499523 0.47736691 0.47439981 0.47678648 0.4

Epoch: 0685 cost = [0.47457053 0.47221587 0.47247058 0.46866598 0.47335067 0.47647686
 0.47277113 0.47206035 0.4765677  0.47523612]
Epoch: 0686 cost = [0.47408126 0.47573523 0.47540604 0.47245071 0.47586854 0.47564449
 0.4725812  0.47435081 0.47661777 0.47588234]
Epoch: 0687 cost = [0.47338938 0.47457411 0.47640867 0.47102198 0.47481744 0.47746043
 0.47359204 0.47427489 0.47613434 0.47467759]
Epoch: 0688 cost = [0.47312019 0.47378357 0.47528544 0.47020545 0.47641783 0.47511779
 0.47356533 0.47339103 0.47533398 0.47387594]
Epoch: 0689 cost = [0.4742229  0.47616844 0.47404273 0.46949117 0.47481469 0.47375076
 0.47374356 0.47363743 0.47549568 0.47411081]
Epoch: 0690 cost = [0.47323661 0.47305942 0.47493059 0.46864892 0.47661886 0.47451612
 0.47262645 0.47468357 0.47654215 0.47408665]
Epoch: 0691 cost = [0.47387976 0.47369049 0.47565291 0.4694687  0.47465782 0.47547243
 0.47375099 0.47457422 0.47659777 0.47385504]
Epoch: 0692 cost = [0.47364205 0.47579758 0.47501923 0.47028902 0.47463777 0

NameError: name '__file__' is not defined